# House Prices competition on Kaggle
- https://www.kaggle.com/c/house-prices-advanced-regression-techniques

In [1]:
import pandas
import matplotlib
import matplotlib.pyplot as plt
import numpy

In [2]:
#import xgboost

In [3]:
import sklearn
import sklearn.model_selection
import sklearn.linear_model
import sklearn.preprocessing
import sklearn.ensemble
import sklearn.svm